# Lab:  Low-Rank Approximation for Movie Recommendations

A common application of low-rank approximation is for recommender systems.  In this lab, we will create a very primitive recommendation system for movies.  Through the lab, you will learn to:

* Represent ratings data as a sparse matrix
* Evaluate the mean absolute error (MAE) using simple movie or user biases
* Create a low-rank model in Tensorflow for predicting the movie rating using `Embedding` layers
* Train the model and optimize the embedding dimension.
* Make predictions on new users.

## Loading the MovieLens Dataset

We first load some common packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

[GroupLens](https://grouplens.org/) is a research organization at the University of Minnesota that has done extensive work in recommendation systems among other topics.  They have excellent datasets on movie recommendations as part of their [MovieLens project](https://movielens.org/).  In this lab, we will use a relatively small dataset, `Movielens 1m` that has 1 million ratings.  If you are interested in continuing research in this area, they have much larger datasets as well.  But, this relatively small one is sufficient to illustrate the basic concepts.

To get the data, go to the webpage:

https://grouplens.org/datasets/movielens/latest/

and download and unzip the files, `ml-1m.zip`.  Alternatively, you can just run the following command which will download unzip the file for you.

In [2]:
import tqdm
import requests
import os
import zipfile

# Set the files names for movies and ratings files
ml_dir = 'ml-1m'
ratings_fn = os.path.join(ml_dir,'ratings.dat')
movies_fn = os.path.join(ml_dir,'movies.dat')

def download_file(src_url, dst_fn):
    
    if os.path.exists(dst_fn):
        print('File %s already exists' % dst_fn)
        return
    
    print('Downloading %s' % dst_fn)
    
    # Streaming, so we can iterate over the response.
    r = requests.get(src_url, stream=True)

    # Total size in MB.
    total_size = int(r.headers.get('content-length', 0)); 
    block_size = 1024
    wrote = 0 
    with open(dst_fn, 'wb') as f:
        with tqdm.tqdm_notebook(total=total_size//block_size, unit='kB', 
                           unit_scale=True, unit_divisor=1024) as pbar:
            for data in r.iter_content(block_size):
                wrote = wrote + len(data)
                pbar.update(1)
                f.write(data)
    if total_size != 0 and wrote != total_size:
        print("ERROR, something went wrong") 

# Test if all files are downloaded
files_exists = False
if os.path.exists(ml_dir):
    if os.path.exists(ratings_fn) and os.path.exists(movies_fn):
        files_exists = True

if files_exists:
    print('Files %s and %s already downloaded' % (ratings_fn, movies_fn))

else:
    # First download the zip file if needed
    src_url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
    dst_fn = 'ml-1m.zip'
    download_file(src_url, dst_fn)
    
    # Then, unzip the file
    print('Unzipping %s...' % dst_fn)
    zip_ref = zipfile.ZipFile(dst_fn, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
    print('Unzip completed')

<ipython-input-2-f2d8d1dba5e0>:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm.tqdm_notebook(total=total_size//block_size, unit='kB',



Unzipping ml-1m.zip...
Unzip completed


Read the movies files with the `read_csv` command.  Print the first 5 entries of the dataframe.  You will see that the file has a list of movies.  Each movie has a `movieId` and `title`.

In [3]:
# TODO:  Read the movies 
import os

# TODO:  Use the movies.head() to display the first 5 entries


Extract the following columns from the `movies` dataframe:
*  Extract the `movieId` column, convert to an `np.array` and store in `movie_ids`
*  Extract the `title` column, convert to a list (using `.tolist()`) and store in `titles`

In [4]:
# TODO:
# movie_id = ...
# titles = ...

The following function returns the string of a movie title, given its movie id.

In [5]:
def get_movie_title(movie_id):
    I = np.where(movie_ids == movie_id)[0]
    if len(I) == 0:
        return 'unknown'
    else:
        return titles[I[0]]

Load the `ratings.dat` file into a `pandas` dataframe `ratings`.  Use the `head` method to print the first five rows of the dataframe.  This is a large file, so it may take a minute to read in.

In [5]:
# TODO
# ratings = ...
#ratings = pd.read_csv(...)

Extract three columns from the `ratings` dataframe:

* Set `user` to `ratings['userId']`,
* Set `movie` to `ratings['movieId']`
* Set `y` to `ratings['rating']`

Convert to each to an `np.array`.  Print:

* Total number of movies (the maximum movie index)
* Total number of users
* Total number of ratings
* The average fraction of movies rated per user

You should see that only a small fraction of the movies are rated by each user.

In [6]:
# TODO
#   user = ...
#   movie = ...
#   y = ...

Our goal will be to predict the rating `y` from the indices `movie` and `user`.  We need to split the data into training and test.
Create training and test data of the form:

* Training data:  `Xtr = [usertr, movietr]` and `ytr` for approximately 75% of the samples.
* Test data:  `Xts = [userts, moviets]` and `yts` for approximately 25% of the samples.

In [7]:
# TODO
#   Xtr, ytr = ...
#   Xts, yts = ...

## Simple Rating Prediction Based on Average Rating

Before we try to perform a complex algorithm for predicting a movie rating, we will first compute some simple statistics to get you familar with the data set.  First, compute the average movie rating across all movies in the training data set.

In [8]:
# TODO

Next, find the average rating per movie id. For each movie id, `i` compute `ymean[i]`, the average rating for that movie in the training data set and `ycnt[i]`, the number of ratings the movie had.  If `ycnt[i]==0`, set `ymean[i]=y0`, where `y0` is the average overall rating. 

You will want to think about how you do this computation efficiently since the data set `ytr` has a large number of entries.  Make you sure you go over the entries in `ytr` only once.  Even if you do it efficiently, it will take a minute, so you may want to add a progress bar.

In [9]:
# TODO
# ymean = ...

Print all the movies that had an average rating over 4.8.  Print their titles, the average rating and the number of ratings they had.  You will see that most of the movies with very high ratings had very few ratings.

In [10]:
# TODO

Now, for each `i` in the test data set, compute `yhat[i]` to be the mean rating for the movie in rating `i`.  Find the average value `|yhat[i]-yts[i]|`.  This is called the *mean absolute error* or MAE and is a common metric in evaluating recommendation predictions.  If you did everything correctly, you should get an MAE ~= 0.78.  That means that simply using the average movie rating by users will predict the rating of another user within 0.78 on average.

In [11]:
# TODO
#   yhat = ...
#   mae = ...

## Building a Neural Network Recommender
We now build a neural network for predicting the ratings.  First, we load the necessary packages.

In [18]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Dense, Flatten, Add, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

We can now create a neural network in Tensorflow as follows:

*  Set the embedding dimension to `emb_dim=4`.  
*  Let `userid_in` and `movieid_in` be the input user and movie indices.  These can be created in Tensorflow with `Input` layers with `shape = (1,)`. 
*  The user index generates a bias `user_bias`.  Use an `Embedding` layer with `output_dim=1` followed by a `Flatten` layer.  
*  The user index also generates a weight `user_wt`.  Use a second `Embedding` layer with `output_dim=emb_dim` followed by a `Flatten` layer.  
*  The movie index generates biases `movie_bias` and `movie_wt` similar to the user bias.
*  We then make the rating prediction with `yhat = Dot(user_wt, movie_wt) + user_bias + movie_bias`. 
*  Optionally, you can add bias and weight regularization, although I found these did not help significantly.
*  Set the model to `mod = Model([userid_in, movieid_in], yhat)`.

Print a summary of the model `mod.summary()`. 


In [12]:
# TODO:
#    K.clear_session()
#    mod = ...
#    mod.summary()


Compile the model with `Adam` optimizer with a learning rate of `0.01` (I found these numbers to work out well).  Use the 
`'mean_absolute_error'` loss.  Then fit the model with 8 epochs.  Use a batch size of 1000.  

In [13]:
# TODO
#    opt = ...
#    mod.compile(...)
#    mod.fit(...)

Print the training and test loss as a function of the epochs.  If you did it correctly the final test loss should be around  0.71 and the training loss should be 0.68.  This is a little better than the MAE you get just using the average movie rating.

In [14]:
# TODO

## Making Predictions!

Select a random user, `user_id`.  Then, for each movie index, use the model to predict the ratings. Set the predictions to `yhat`.

In [15]:
# TODO
#    user_id = ...
#    yhat = mod.predict(...)

Print the names of the movies with the top 10 predicted ratings for the user as well as the average rating that those movies had.  You will see that the network may predict ratings above 5!  We could have avoided this by limiting the output.

In [16]:
# TODO

## Bonus:  Optimizing the Embedding Dimension

You can try to optimize the embedding dimension.  Try different dimensions from 0 to 8.  

In [17]:
# TODO


Plot the training and test loss as a function of the embedding layer. We see we get a minimum with an embedding dimension around 4 or 5.